In [1]:
import gymnasium as gym

env = gym.make("Pong-v4", render_mode="rgb_array")

A.L.E: Arcade Learning Environment (version 0.8.1+unknown)
[Powered by Stella]
Game console created:
  ROM file:  /Users/Eugene/miniconda3/envs/rl_learning/lib/python3.12/site-packages/AutoROM/roms/pong.bin
  Cart Name: Video Olympics (1978) (Atari)
  Cart MD5:  60e0ea3cbe0913d39803477945e9e5ec
  Display Format:  AUTO-DETECT ==> NTSC
  ROM Size:        2048
  Bankswitch Type: AUTO-DETECT ==> 2K

Running ROM file...
Random seed is -1283115538


In [ ]:
# Create parameters

# Loop for x steps:
  # Preprocess
    # RGB -> binary
    # Downsample

  # 